# 什么是单元测试

单元测试，通俗易懂地讲，就是编写测试来验证某一个模块的功能正确性，一般会指定输入，验证输出是否符合预期。

实际生产环境中，我们会对每一个模块的所有可能输入值进行测试。这样虽然显得繁琐，增加了额外的工作量，但是能够大大提高代码质量，减小 bug 发生的可能性，也更方便系统的维护。

说起单元测试，就不得不提 Python unittest 库，它提供了我们需要的大多数工具。

我们来看下面这个简单的测试，从代码中了解其使用方法：

In [1]:
import unittest

# 将要被测试的排序函数
def sort(arr):
    l = len(arr)
    for i in range(0, l):
        for j in range(i + 1, l):
            if arr[i] >= arr[j]:
                tmp = arr[i]
                arr[i] = arr[j]
                arr[j] = tmp


# 编写子类继承unittest.TestCase
class TestSort(unittest.TestCase):

   # 以test开头的函数将会被测试
   def test_sort(self):
        arr = [3, 4, 1, 5, 6]
        sort(arr)
        # assert 结果跟我们期待的一样
        self.assertEqual(arr, [1, 3, 4, 5, 6])

if __name__ == '__main__':
    ## 如果在Jupyter下，请用如下方式运行单元测试
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    
    ## 如果是命令行下运行，则：
    ## unittest.main()
    


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


我们创建了一个排序函数的单元测试，来验证排序函数的功能是否正确。

1. 首先，我们需要创建一个类TestSort，继承类‘unittest.TestCase’；
2. 然后，在这个类中定义相应的测试函数 test_sort()，进行测试。注意，测试函数要以‘test’开头，而测试函数的内部，通常使用 assertEqual()、assertTrue()、assertFalse() 和 assertRaise() 等 assert 语句对结果进行验证。
3. 最后运行时，如果是在 IPython 或者 Jupyter 环境下，请使用下面这行代码 `unittest.main(argv=['first-arg-is-ignored'], exit=False)`;如果用的是命令行，直接使用 unittest.main() 就可以了

# 单元测试的几个技巧

将会介绍 Python 单元测试的几个技巧，分别是 mock、side_effect 和 patch。这三者用法不一样，但都是一个核心思想，**即用虚假的实现，来替换掉被测试函数的一些依赖项，让我们能把更多的精力放在需要被测试的功能上**。

## mock

mock 是单元测试中最核心重要的一环。mock 的意思，便是通过一个虚假对象，来代替被测试函数或模块需要的对象。

举个例子，比如你要测一个后端 API 逻辑的功能性，但一般后端 API 都依赖于数据库、文件系统、网络等。这样，你就需要通过 mock，来创建一些虚假的数据库层、文件系统层、网络层对象，以便可以简单地对核心后端逻辑单元进行测试。Python mock 则主要使用 mock 或者 MagicMock 对象。



In [2]:
import unittest
from unittest.mock import MagicMock

class A(unittest.TestCase):
    def m1(self):
        val = self.m2()
        self.m3(val)

    def m2(self):
        pass

    def m3(self, val):
        pass

    def test_m1(self):
        a = A()
        a.m2 = MagicMock(return_value="custom_val")
        a.m3 = MagicMock()
        a.m1()
        self.assertTrue(a.m2.called) #验证m2被call过
        a.m3.assert_called_with("custom_val") #验证m3被指定参数call过
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


这段代码中，我们定义了一个类的三个方法 m1()、m2()、m3()。

我们需要对 m1() 进行单元测试，但是 m1() 取决于 m2() 和 m3()。如果 m2() 和 m3() 的内部比较复杂, 你就不能只是简单地调用 m1() 函数来进行测试，可能需要解决很多依赖项的问题。

这一听就让人头大了吧？但是，有了 mock 其实就很好办了。我们可以把 m2() 替换为一个返回具体数值的 value，把 m3() 替换为另一个 mock（空函数）。这样，测试 m1() 就很容易了，我们可以测试 m1() 调用 m2()，并且用 m2() 的返回值调用 m3()。

可能你会疑惑，这样测试 m1() 不是基本上毫无意义吗？

看起来只是象征性地测了一下逻辑呀？ 其实不然，真正工业化的代码，都是很多层模块相互逻辑调用的一个树形结构。单元测试需要测的是某个节点的逻辑功能，mock 掉相关的依赖项是非常重要的。这也是为什么会被叫做单元测试 unit test，而不是其他的 integration test、end to end test 这类。

## Mock Side Effect

mock的函数，属性是可以根据不同的输入，返回不同的数值，而不只是一个return_value。

比如下面这个示例，测试的是输入参数是否为负数，输入小于 0 则输出为 1 ，否则输出为 2。这便是 Mock Side Effect 的用法。

In [8]:
from unittest.mock import MagicMock

def side_effect(arg):
    if arg < 0:
        return 1
    else:
        return 2
    
mock = MagicMock()
mock.side_effect = side_effect

print(mock(-1))
print(mock(1))

1
2


## patch

至于 patch，给开发者提供了非常便利的函数 mock 方法。它可以应用 Python 的 decoration 模式或是 context manager 概念，快速自然地 mock 所需的函数。

In [ ]:
from unittest.mock import patch

@patch('sort')
def test_sort(self, mock_sort):
    ...
    ...


在这个 test 里面，mock_sort 替代 sort 函数本身的存在，所以，我们可以像开始提到的 mock object 一样，设置 return_value 和 side_effect。

另一种 patch 的常见用法，是 mock 类的成员函数，这个技巧我们在工作中也经常会用到，比如说一个类的构造函数非常复杂，而测试其中一个成员函数并不依赖所有初始化的 object。它的用法如下：

In [ ]:
with patch.object(A, '__init__', lambda x: None):
      …

在 with 语句里面，我们通过 patch，将 A 类的构造函数 mock 为一个 do nothing 的函数，这样就可以很方便地避免一些复杂的初始化（initialization）。

单元测试的核心还是 mock，mock 掉依赖项，测试相应的逻辑或算法的准确性。

## 高质量单元测试的关键

一个高质量的单元测试，应该特别关注下面两点。

### Test Coverage

它是衡量代码中语句被 cover 的百分比。可以说，提高代码模块的 Test Coverage，基本等同于提高代码的正确性。

我们可以用 Python 的 coverage tool 来衡量 Test Coverage，并且显示每个模块为被 coverage 的语句。如果你想了解更多更详细的使用，可以点击这个链接来学习：https://coverage.readthedocs.io/en/v4.5.x/

### 模块化

高质量单元测试，不仅要求我们提高 Test Coverage，尽量让所写的测试能够 cover 每个模块中的每条语句；还要求我们从测试的角度审视 codebase，去思考怎么模块化代码，以便写出高质量的单元测试。

下面这个函数，对一个数组进行处理，并返回新的数组：

In [ ]:
def work(arr):
    # pre process
    ...
    ...
    # sort
    l = len(arr)
    for i in range(0, l):
        for j in range(i + 1, j):
            if arr[i] >= arr[j]:
                tmp = arr[i]
                arr[i] = arr[j]
                arr[j] = tmp
    # post process
    ...
    ...
    return arr


这段代码的大概意思是，先有个预处理，再排序，最后再处理一下然后返回。如果现在要求你，给这个函数写个单元测试，你是不是会一筹莫展呢？

毕竟，这个函数确实有点儿复杂，以至于你都不知道应该是怎样的输入，并要期望怎样的输出。这种代码写单元测试是非常痛苦的，更别谈 cover 每条语句的要求了。

所以，正确的测试方法，应该是先模块化代码，写成下面的形式:

In [ ]:
def preprocess(arr):
    ...
    ...
    return arr

def sort(arr):
    ...
    ...
    return arr

def postprocess(arr):
    ...
    return arr

def work(self):
    arr = preprocess(arr)
    arr = sort(arr)
    arr = postprocess(arr)
    return arr


接着再进行相应的测试，测试三个子函数的功能正确性；然后通过 mock 子函数，调用 work() 函数，来验证三个子函数被 call 过。

In [ ]:
from unittest.mock import patch

def test_preprocess(self):
    ...
    
def test_sort(self):
    ...
    
def test_postprocess(self):
    ...
    
@patch('%s.preprocess')
@patch('%s.sort')
@patch('%s.postprocess')
def test_work(self,mock_post_process, mock_sort, mock_preprocess):
    work()
    self.assertTrue(mock_post_process.called)
    self.assertTrue(mock_sort.called)
    self.assertTrue(mock_preprocess.called)


这样一来，通过重构代码就可以使单元测试更加全面、精确，并且让整体架构、函数设计都美观了不少。